# Режет файл с разметкой (GenBank'a) на файлы по хромосомам

In [2]:
import pandas as pd
import subprocess 
from gtfparse import read_gtf

делает из GTF с разметкой датафрейм, упорядочивает его по хромосомам и по началу-концу фичей, ищет бинарным поском границы хромосом и режет по ним. Полученное пишется в файлы по хромосомам


(можно проще, но когда писалось я об этом не знал).

In [13]:
#subprocess.call('rsync rsync://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.39_GRCh38.p13/GCF_000001405.39_GRCh38.p13_feature_table.txt.gz ~/Documents/CourseWork/genomes/Annotation/feature_table_38.txt.gz', shell=True)
#subprocess.call(['gunzip ~/Documents/CourseWork/genomes/Annotation/*.gz'], shell=True)
#subprocess.call(['sort -t$"\t" -k2 feature_table_38.txt > sort_feature_table_38.tsv'], shell=True)
#subprocess.call(['rm feature_table_38.txt'], shell=True)

In [14]:
feature_table = read_gtf("/Users/pitikov_egor/Documents/CourseWork/genomes/Annotation/gencode.v35.annotation.gtf")
feature_table.sort_values(by=['seqname', 'start', 'end'], kind='mergesort').reset_index(drop=True, inplace=True)

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'hgnc_id', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'tag', 'havana_transcript', 'exon_number', 'exon_id', 'ont', 'protein_id', 'ccdsid']


In [15]:
#########subprocess.call(['rm /Users/pitikov_egor/Documents/CourseWork/genomes/Annotation/gencode.v35.annotation.gtf'], shell=True)

0

In [26]:
feature_table = pd.read_csv('/Users/pitikov_egor/Documents/CourseWork/genomes/Annotation/genes.tsv')

In [27]:
feature_table

,Unnamed: 0,seqname,source,feature,start,end,score,strand,frame,gene_id,...,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid
0,0,chr1,HAVANA,gene,11869,14409,NaN,+,0,ENSG00000223972.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12,chr1,HAVANA,gene,14404,29570,NaN,-,0,ENSG00000227232.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25,chr1,ENSEMBL,gene,17369,17436,NaN,-,0,ENSG00000278267.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28,chr1,HAVANA,gene,29554,31109,NaN,+,0,ENSG00000243485.5,...,NaN,NaN,NaN,ncRNA_host,NaN,NaN,NaN,NaN,NaN,NaN
4,36,chr1,ENSEMBL,gene,30366,30503,NaN,+,0,ENSG00000284332.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60655,3014836,chrM,ENSEMBL,gene,14149,14673,NaN,-,0,ENSG00000198695.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60656,3014841,chrM,ENSEMBL,gene,14674,14742,NaN,-,0,ENSG00000210194.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60657,3014844,chrM,ENSEMBL,gene,14747,15887,NaN,+,0,ENSG00000198727.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60658,3014849,chrM,ENSEMBL,gene,15888,15953,NaN,+,0,ENSG00000210195.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
feature_table['seqname'] = feature_table.astype(str)

In [29]:
def binary_find (array, target):
    left = 0
    right = len(array)-1
    while (right-left)!=1:
        if array[(right+left)//2] > target:
            right = (right+left)//2
        elif array[(right+left)//2] < target: 
            left = (right+left)//2
    return right

In [30]:
chr_names=['chrM', 'chrX', 'chrY']
for i in range(1, 23):
    chr_names.append('chr%s' % (i))
chr_names.sort()

In [31]:
coordinates={}  
for i in range(len(chr_names)):
    a = binary_find(feature_table['seqname'], chr_names[i]+' ')
    coordinates[chr_names[i]] = [binary_find(feature_table['seqname'], chr_names[i-1]+' '), binary_find(feature_table['seqname'], chr_names[i]+' ')]                
coordinates[chr_names[0]][0] = 0
coordinates

{'chr1': [0, 60659],
 'chr10': [60659, 60659],
 'chr11': [60659, 60659],
 'chr12': [60659, 60659],
 'chr13': [60659, 60659],
 'chr14': [60659, 60659],
 'chr15': [60659, 60659],
 'chr16': [60659, 60659],
 'chr17': [60659, 60659],
 'chr18': [60659, 60659],
 'chr19': [60659, 60659],
 'chr2': [60659, 60659],
 'chr20': [60659, 60659],
 'chr21': [60659, 60659],
 'chr22': [60659, 60659],
 'chr3': [60659, 60659],
 'chr4': [60659, 60659],
 'chr5': [60659, 60659],
 'chr6': [60659, 60659],
 'chr7': [60659, 60659],
 'chr8': [60659, 60659],
 'chr9': [60659, 60659],
 'chrM': [60659, 60659],
 'chrX': [60659, 60659],
 'chrY': [60659, 60659]}

In [32]:
coordinates={}  
for i in range(len(chr_names)):
    a = binary_find(feature_table['seqname'], chr_names[i]+' ')
    coordinates[chr_names[i]] = [binary_find(feature_table['seqname'], chr_names[i-1]+' '), binary_find(feature_table['seqname'], chr_names[i]+' ')]                
coordinates[chr_names[0]][0] = 0
coordinates

{'chr1': [0, 60659],
 'chr10': [60659, 60659],
 'chr11': [60659, 60659],
 'chr12': [60659, 60659],
 'chr13': [60659, 60659],
 'chr14': [60659, 60659],
 'chr15': [60659, 60659],
 'chr16': [60659, 60659],
 'chr17': [60659, 60659],
 'chr18': [60659, 60659],
 'chr19': [60659, 60659],
 'chr2': [60659, 60659],
 'chr20': [60659, 60659],
 'chr21': [60659, 60659],
 'chr22': [60659, 60659],
 'chr3': [60659, 60659],
 'chr4': [60659, 60659],
 'chr5': [60659, 60659],
 'chr6': [60659, 60659],
 'chr7': [60659, 60659],
 'chr8': [60659, 60659],
 'chr9': [60659, 60659],
 'chrM': [60659, 60659],
 'chrX': [60659, 60659],
 'chrY': [60659, 60659]}

In [33]:
for chromosome in chr_names[:-1]:
    feature_table[coordinates[chromosome][0]:coordinates[chromosome][1]].to_csv('~/Documents/CourseWork/genomes/Annotation/chr_features/%s_genes.tsv' % (chromosome) , sep='\t', index=False)
feature_table[coordinates[chr_names[-1]][0]:coordinates[chr_names[-1]][1]+1].to_csv('~/Documents/CourseWork/genomes/Annotation/chr_features/%s_genes.tsv' % (chr_names[-1]), sep='\t', index=False)